# Import Modules

In [1]:
import tensorflow as tf
from google.colab import files
import numpy as np
!pip install librosa
import librosa
import numpy as np

# Mount Google Drive for file support

In [2]:

# from google.colab import files
# uploaded = files.upload()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


I have mounted my google drive to use the wav files from it. Please change the path to local directory in order to run the code and download the files in the current working directory.

Anopther Option is to uncomment the upload button code and upload files individually from the local system.

# Load & Convert Wav files to respective Spectograms

In [0]:
#S and X are two matrices of size 513x2459  Short time fourier Transform
s, sr=librosa.load('/content/gdrive/My Drive/train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr=librosa.load('/content/gdrive/My Drive/train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)

s1, sr=librosa.load('/content/gdrive/My Drive/test_x_01.wav', sr=None)
S1 =librosa.stft(s1, n_fft=1024, hop_length=512)
s2, sr=librosa.load('/content/gdrive/My Drive/test_x_02.wav', sr=None)
S2 =librosa.stft(s2, n_fft=1024, hop_length=512)
# print(S1.shape, S2.shape)
# print(S.shape, X.shape)

# Calculate Magnitude

In [0]:
S_abs=np.transpose(np.abs(S))
X_abs=np.transpose(np.abs(X))

In [0]:
#Declaring Placeholder Variables
x = tf.placeholder(tf.float32, [None, 513])
y = tf.placeholder(tf.float32, [None, 513])

epochs = 1000
batch_size = 250

# Initialise Weights & Biases for the Network

In [0]:
# W1 = tf.Variable(tf.random_normal([513, 1024], stddev=0.03), name='W1')
# b1 = tf.Variable(tf.random_normal([1024]), name='b1')
# # and the weights connecting the hidden layer to the output layer
# W2 = tf.Variable(tf.random_normal([1024, 1024], stddev=0.03), name='W2')
# b2 = tf.Variable(tf.random_normal([1024]), name='b2')

# W3 = tf.Variable(tf.random_normal([1024, 513], stddev=0.03), name='W3')
# b3 = tf.Variable(tf.random_normal([513]), name='b3')

W1=tf.Variable(tf.initializers.he_normal(seed=None)((513, 1024)))
b1 = tf.Variable(tf.random_normal([1024]), name='b1')

W2=tf.Variable(tf.initializers.he_normal(seed=None)((1024, 1000)))
b2 = tf.Variable(tf.random_normal([1000]), name='b2')

W3=tf.Variable(tf.initializers.he_normal(seed=None)((1000, 1024)))
b3 = tf.Variable(tf.random_normal([1024]), name='b3')

W4=tf.Variable(tf.initializers.he_normal(seed=None)((1024, 513)))
b4 = tf.Variable(tf.random_normal([513]), name='b4')

# W4 = tf.Variable(tf.random_normal([1024, 513], stddev=0.03), name='W4')
# b4 = tf.Variable(tf.random_normal([513]), name='b4')



# Compose the Network Architecture

In [0]:
#Putting together Layers
hidden_1 = tf.add(tf.matmul(x, W1), b1)
hidden_1 = tf.nn.relu(hidden_1)
hidden_2 = tf.add(tf.matmul(hidden_1, W2), b2)
hidden_2 = tf.nn.relu(hidden_2)
hidden_3 = tf.add(tf.matmul(hidden_2, W3), b3)
hidden_3 = tf.nn.relu(hidden_3)


y_ = tf.nn.relu(tf.add(tf.matmul(hidden_3, W4), b4))

# Define Loss Function

In [0]:
cost= tf.losses.mean_squared_error(y,y_, weights=1.0)
loss_function = tf.reduce_mean(cost, name = 'loss_function')
optimize = tf.train.AdamOptimizer(0.0001).minimize(loss_function)

# Training Network

In [20]:
init_op = tf.global_variables_initializer()
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(init_op)

batch=int(X_abs.shape[0]/batch_size)

for epoch in range(epochs):
    loss = 0
    start = 0
    for _ in range(batch):
        end = start +batch_size
        if end > X_abs.shape[0]:end = X_abs.shape[0]
        x_batch = X_abs[start:end]
        y_batch = S_abs[start:end]
        start = end + 1
        _ = sess.run(optimize, feed_dict={x: x_batch, y: y_batch})
        c= sess.run(cost, feed_dict={x: x_batch, y: y_batch})
        loss += c
        
    print('Epoch ',epoch, 'loss: ', loss)
    
    save_path = saver.save(sess, "/content/gdrive/My Drive/dl02_model.ckpt")
        
sess.close()   
        

Epoch  0 loss:  9.045808881521225
Epoch  1 loss:  1.8065841645002365
Epoch  2 loss:  0.8350165784358978
Epoch  3 loss:  0.7110746577382088
Epoch  4 loss:  0.6367659345269203
Epoch  5 loss:  0.5752289034426212
Epoch  6 loss:  0.5268498286604881
Epoch  7 loss:  0.4872427601367235
Epoch  8 loss:  0.4549154043197632
Epoch  9 loss:  0.42887023463845253
Epoch  10 loss:  0.40779533982276917
Epoch  11 loss:  0.39040587842464447
Epoch  12 loss:  0.37628900073468685
Epoch  13 loss:  0.3647633586078882
Epoch  14 loss:  0.35527439415454865
Epoch  15 loss:  0.34755510464310646
Epoch  16 loss:  0.3409371804445982
Epoch  17 loss:  0.33449243940413
Epoch  18 loss:  0.31812382116913795
Epoch  19 loss:  0.2936648391187191
Epoch  20 loss:  0.2809950262308121
Epoch  21 loss:  0.26867179945111275
Epoch  22 loss:  0.2552047111093998
Epoch  23 loss:  0.24700164794921875
Epoch  24 loss:  0.2412311714142561
Epoch  25 loss:  0.23697298392653465
Epoch  26 loss:  0.23361879028379917
Epoch  27 loss:  0.23076199274

# Testing Network

In [39]:

sess = tf.InteractiveSession()
saver.restore(sess, "/content/gdrive/My Drive/dl02_model.ckpt")
First_cleaned = sess.run(y_, feed_dict = {x: np.abs(S1.T)})
Second_cleaned = sess.run(y_, feed_dict = {x: np.abs(S2.T)})
Original_cleaned=sess.run(y_, feed_dict = {x: np.abs(X.T)})
sess.close()

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/dl02_model.ckpt


# Recover Speech Spectogram from test signal & apply Inverse STFT

In [0]:
temp1=np.abs(S1.T)
Cleaned1 = (S1/temp1.T)* First_cleaned.T
recons_01 = librosa.istft(Cleaned1,  hop_length=512)
temp2=np.abs(S2.T)
Cleaned2 = (S2/temp2.T)* Second_cleaned.T
recons_02 = librosa.istft(Cleaned2, hop_length=512)
temp3=np.abs(X.T)
CleanedO = (X/temp3.T)* Original_cleaned.T
recons_03 = librosa.istft(CleanedO, hop_length=512)




# Write and Download the output cleaned output WAV file for test samples

In [0]:
librosa.output.write_wav('test_s_01_recons.wav', recons_01, sr)
librosa.output.write_wav('test_s_02_recons.wav', recons_02, sr)
librosa.output.write_wav('Original_recons.wav', recons_03, sr)


Run Individual Cells to download files.

In [0]:

files.download( "test_s_01_recons.wav" )


In [0]:

files.download( "test_s_02_recons.wav" ) 


In [0]:

files.download( "Original_recons.wav" ) 
